In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import sklearn 

In [2]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv" , encoding='ISO-8859-1')

In [3]:
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
data['0'].value_counts()

4    800000
0    799999
Name: 0, dtype: int64

In [5]:
data['NO_QUERY'].value_counts()

NO_QUERY    1599999
Name: NO_QUERY, dtype: int64

In [6]:
data['_TheSpecialOne_'].value_counts()[:10]

lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
mcraddictal        276
tsarnick           248
what_bugs_u        246
Karen230683        238
DarkPiano          236
Name: _TheSpecialOne_, dtype: int64

In [7]:
data = data.rename(columns={"0":"label" , "Mon Apr 06 22:19:45 PDT 2009":"created_at","NO_QUERY":"status",
                     "_TheSpecialOne_":"user_name" ,
                     "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D":"text", 
                    "1467810369":"id"})

In [8]:
data = data.rename(columns = {"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D":"text"})

In [9]:
data.head()

,label,id,created_at,status,user_name,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
print(len(data))

1599999


In [11]:
data = data.drop_duplicates()
print(len(data))

1599999


In [12]:
data = data.drop(columns= ["status"])
data.head()

,label,id,created_at,user_name,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,@Kwesidei not the whole crew


In [13]:
import nltk

In [135]:
d0 = data[data['label'] == 0 ][:200000]
d4 = data[data['label'] == 4][:200000]
dd = pd.concat([d0,d4]).reset_index()
X = dd['text'].values
Y = dd['label'].values

In [91]:
# bag of words
import spacy
nlp = spacy.load("en_core_web_sm")

In [136]:
# bag of words 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
vectorizer_bag = CountVectorizer(tokenizer=word_tokenize , stop_words=['the'])

In [104]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rayan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [144]:
from sklearn.model_selection import train_test_split
x_train_ , x_test , y_train_ , y_test = train_test_split(X , Y , test_size=0.3)

In [145]:
x_train , x_val , y_train , y_val = train_test_split(x_train_ , y_train_ , test_size=0.3)

In [146]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
model_naive_bag = Pipeline([
    ('vectorizer' , CountVectorizer(min_df=0.0 , lowercase=False)),
    ('nb' , MultinomialNB())
])
model_naive_bag.fit(x_train , y_train)

Pipeline(steps=[('vectorizer', CountVectorizer(lowercase=False, min_df=0.0)),
                ('nb', MultinomialNB())])

In [150]:
print("accuracy naive BOW : " , model_naive_bag.score(x_val , y_val))

accuracy naive BOW :  0.7662380952380953


In [152]:
from sklearn.ensemble import AdaBoostClassifier
model_ada_bag = Pipeline([
    ('vect_ada_bag' , CountVectorizer(min_df=0.0 , lowercase=False)),
    ('ada_bag' , AdaBoostClassifier())
])
model_ada_bag.fit(x_train , y_train)

Pipeline(steps=[('vect_ada_bag', CountVectorizer(lowercase=False, min_df=0.0)),
                ('ada_bag', AdaBoostClassifier())])

In [153]:
print("accuracy AdaBoost BOW : " , model_ada_bag.score(x_val , y_val))

accuracy AdaBoost BOW :  0.6707738095238095


In [156]:
from sklearn.linear_model import LogisticRegression
model_logreg_bag = Pipeline([
    ('vect_logreg_bag' , CountVectorizer(min_df=0.0 , lowercase=False)),
    ('logreg_bag' , LogisticRegression())
])
model_logreg_bag.fit(x_train,y_train)

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect_logreg_bag',
                 CountVectorizer(lowercase=False, min_df=0.0)),
                ('logreg_bag', LogisticRegression())])

In [158]:
print("accuracy logreg BOW : " , model_logreg_bag.score(x_val , y_val))

accuracy logreg BOW :  0.7811904761904762


In [134]:
# TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=word_tokenize)

In [148]:
model_naive_tf = Pipeline([
    ('vec_tf' , TfidfVectorizer(tokenizer=word_tokenize , min_df=0.0 , lowercase=False)) , 
    ('nb_tf' , MultinomialNB())
])
model_naive_tf.fit(x_train , y_train)

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vec_tf',
                 TfidfVectorizer(lowercase=False, min_df=0.0,
                                 tokenizer=<function word_tokenize at 0x000001B37D3C8EE0>)),
                ('nb_tf', MultinomialNB())])

In [151]:
print("accuracy naive tfidf : " , model_naive_tf.score(x_val , y_val))

accuracy naive tfidf :  0.7638452380952381


In [163]:
model_ada_tf = Pipeline([
    ('vect_ada_tf' , TfidfVectorizer(tokenizer=word_tokenize , min_df=0.0 , lowercase=False)),
    ('ada_tf' , AdaBoostClassifier())
])

model_ada_tf.fit(x_train, y_train)

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect_ada_tf',
                 TfidfVectorizer(lowercase=False, min_df=0.0,
                                 tokenizer=<function word_tokenize at 0x000001B37D3C8EE0>)),
                ('ada_tf', AdaBoostClassifier())])

In [164]:
print("accuracy AdaBoost tfidf : " , model_ada_tf.score(x_val , y_val))

accuracy AdaBoost tfidf :  0.6978452380952381


In [165]:
model_logreg_tf = Pipeline([
    ('vect_logreg_tf' , TfidfVectorizer(tokenizer=word_tokenize , min_df=0.0 , lowercase=False)),
    ('logreg_tf'  , LogisticRegression())
])
model_logreg_tf.fit(x_train , y_train)

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect_logreg_tf',
                 TfidfVectorizer(lowercase=False, min_df=0.0,
                                 tokenizer=<function word_tokenize at 0x000001B37D3C8EE0>)),
                ('logreg_tf', LogisticRegression())])

In [166]:
print("accuracy logreg tfidf : " , model_logreg_tf.score(x_val,y_val))

accuracy logreg tfidf :  0.7898690476190476


In [168]:
# با توجه به نتایج به دست آمده ، آموزش مدل
# logistic regression 
# با روش
# tfidf
# بهترین نتیجه را میدهد
# پس برای دیتای تست ، از همین روند استفاده میکنیم

In [167]:
print("accuracy logreg tfidf on test data : " , model_logreg_tf.score(x_test,y_test))

accuracy logreg tfidf on test data :  0.791375


In [172]:
from transformers import BertModel , BertConfig
bert_configuration = BertConfig()
bert_model = BertModel(bert_configuration)
bert_configuration = bert_model.config

In [173]:
from transformers import AutoTokenizer , BertModel , TFBertTokenizer
tokenizer_pre = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rayan\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [190]:
print(inp_bert[0])

TypeError: 'tokenizers.Encoding' object is not subscriptable

In [191]:
from transformers import TFBertModel , TFBertTokenizer
tf_bert_token = TFBertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
inp_bert = tf_(list(x_train) , return_tensors='tf')
outputs = bert_model(inp_bert)

ImportError: 
TFBertTokenizer requires the tensorflow_text library but it was not found in your environment. You can install it with pip as
explained here: https://www.tensorflow.org/text/guide/tf_text_intro.
Please note that you may need to restart your runtime after installation.


In [193]:
from tensorflow import keras_nlp

features = x_train
labels = y_train

# Pretrained classifier.
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_en_uncased",
    num_classes=2,
)
classifier.fit(x=features, y=labels, batch_size=2)
bert_pred_val = classifier.predict(x=x_val, batch_size=2)

ImportError: cannot import name 'keras_nlp' from 'tensorflow' (C:\Users\Rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\__init__.py)